# Data Cleaning

This notebook loads the raw encounter export and cleans it before we calculate revenue.

Goals:
- standardize column names
- keep only closed + billable encounters
- remove junk rows (repeated headers)
- export a clean dataset for the next notebook

## Load the data

Read in the raw CSV file and take a quick look at the structure.

In [22]:
import pandas as pd
from pathlib import Path

# Update the filename if yours is different
RAW_PATH = Path("..") / "data" / "sample_monthly_data.csv"

df_raw = pd.read_csv(RAW_PATH)

print("Raw columns:")
print(df_raw.columns.tolist())

df_raw.head()

Raw columns:
['Sex', 'City', 'County', 'Patient Category', 'Encounter Facility', 'POS', 'Encounter Type', 'Visit Date', 'Visit Time', 'Race', 'Patient Status', 'Encounter / Service Provider', 'CPT / Revenue', 'Duration (Min)', 'Is Billable', 'Encounter Status']


,Sex,City,County,Patient Category,Encounter Facility,POS,Encounter Type,Visit Date,Visit Time,Race,Patient Status,Encounter / Service Provider,CPT / Revenue,Duration (Min),Is Billable,Encounter Status
0,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,11:00 AM,African American,X Discharged,KW,T1012,23,Yes,Closed
1,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,12:45 PM,White,X Discharged,KW,T1012,30,Yes,Closed
2,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,2:00 PM,Other Race,X Discharged,KW,T1012,31,Yes,Closed
3,Male,Potterville,Eaton County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,2:45 PM,White,X Discharged,KW,T1012,19,Yes,Closed
4,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,4:15 PM,White,X Discharged,KW,T1012,30,Yes,Closed


## Standardize column names

The raw export uses spaces and slashes in headers (ex: "CPT / Revenue").
We normalize headers so downstream code is consistent.

In [23]:
df = df_raw.copy()

df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("/", "_")
      .str.replace("(", "")
      .str.replace(")", "")
)

print("Standardized columns:")
print(df.columns.tolist())

df.head()

Standardized columns:
['sex', 'city', 'county', 'patient_category', 'encounter_facility', 'pos', 'encounter_type', 'visit_date', 'visit_time', 'race', 'patient_status', 'encounter___service_provider', 'cpt___revenue', 'duration_min', 'is_billable', 'encounter_status']


,sex,city,county,patient_category,encounter_facility,pos,encounter_type,visit_date,visit_time,race,patient_status,encounter___service_provider,cpt___revenue,duration_min,is_billable,encounter_status
0,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,11:00 AM,African American,X Discharged,KW,T1012,23,Yes,Closed
1,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,12:45 PM,White,X Discharged,KW,T1012,30,Yes,Closed
2,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,2:00 PM,Other Race,X Discharged,KW,T1012,31,Yes,Closed
3,Male,Potterville,Eaton County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,2:45 PM,White,X Discharged,KW,T1012,19,Yes,Closed
4,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,4:15 PM,White,X Discharged,KW,T1012,30,Yes,Closed


## Rename key columns

We rename important fields so they match what the revenue module expects:
- encounter_date
- cpt_code
- duration_min
- is_billable
- encounter_status

In [24]:
df = df.rename(columns={
    "visit_date": "encounter_date",
    "cpt___revenue": "cpt_code",
    "duration_min": "duration_min",
    "is_billable": "is_billable",
    "encounter_status": "encounter_status"
})

df.head()

,sex,city,county,patient_category,encounter_facility,pos,encounter_type,encounter_date,visit_time,race,patient_status,encounter___service_provider,cpt_code,duration_min,is_billable,encounter_status
0,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,11:00 AM,African American,X Discharged,KW,T1012,23,Yes,Closed
1,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,12:45 PM,White,X Discharged,KW,T1012,30,Yes,Closed
2,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,2:00 PM,Other Race,X Discharged,KW,T1012,31,Yes,Closed
3,Male,Potterville,Eaton County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,2:45 PM,White,X Discharged,KW,T1012,19,Yes,Closed
4,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,4/5/2023,4:15 PM,White,X Discharged,KW,T1012,30,Yes,Closed


## Convert data types

Make sure:
- encounter_date is datetime
- duration_min is numeric

In [25]:
df["encounter_date"] = pd.to_datetime(df["encounter_date"], errors="coerce")
df["duration_min"] = pd.to_numeric(df["duration_min"], errors="coerce")

df.info()

<class 'pandas.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   sex                           2337 non-null   str           
 1   city                          2337 non-null   str           
 2   county                        2335 non-null   str           
 3   patient_category              24 non-null     str           
 4   encounter_facility            2337 non-null   str           
 5   pos                           2337 non-null   str           
 6   encounter_type                2337 non-null   str           
 7   encounter_date                2313 non-null   datetime64[us]
 8   visit_time                    2337 non-null   str           
 9   race                          2337 non-null   str           
 10  patient_status                2337 non-null   str           
 11  encounter___service_provider  2337 non-nu

## Clean + filter

Your export marks completed encounters as "Closed" and billable as "Yes".

Also, some exports accidentally include repeated header rows inside the file
(like rows that literally say "Encounter Status" or "Is Billable").
We remove those too.

In [26]:
# Normalize values for filtering
status = df["encounter_status"].astype(str).str.strip().str.lower()
billable = df["is_billable"].astype(str).str.strip().str.lower()

# Remove repeated header rows that show up as values
df = df[~status.eq("encounter status")]
df = df[~billable.eq("is billable")]

# Keep only closed + billable
df = df[status.eq("closed") & billable.eq("yes")]

# Drop rows missing required fields
df = df.dropna(subset=["encounter_date", "cpt_code"]).reset_index(drop=True)

print("Cleaned rows:", len(df))
df.head()

Cleaned rows: 2312


/tmp/ipykernel_16541/2304004754.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~billable.eq("is billable")]
/tmp/ipykernel_16541/2304004754.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[status.eq("closed") & billable.eq("yes")]


,sex,city,county,patient_category,encounter_facility,pos,encounter_type,encounter_date,visit_time,race,patient_status,encounter___service_provider,cpt_code,duration_min,is_billable,encounter_status
0,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,2023-04-05,11:00 AM,African American,X Discharged,KW,T1012,23.0,Yes,Closed
1,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,2023-04-05,12:45 PM,White,X Discharged,KW,T1012,30.0,Yes,Closed
2,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,2023-04-05,2:00 PM,Other Race,X Discharged,KW,T1012,31.0,Yes,Closed
3,Male,Potterville,Eaton County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,2023-04-05,2:45 PM,White,X Discharged,KW,T1012,19.0,Yes,Closed
4,Male,Lansing,Ingham County,NaN,Men's Safe Harbor,Office,PRC Individual - CCAR,2023-04-05,4:15 PM,White,X Discharged,KW,T1012,30.0,Yes,Closed


## Quick checks

Just a couple sanity checks:
- total row count
- most common CPT codes

In [27]:
print("Final row count:", len(df))
print("\nTop CPT codes:")
print(df["cpt_code"].value_counts().head(20))

Final row count: 2312

Top CPT codes:
cpt_code
H0038    1804
T1012     508
Name: count, dtype: int64


## Export cleaned dataset

This file will be used in Notebook 02 for revenue calculation.

In [28]:
OUT_PATH = Path("..") / "data" / "clean_encounters.csv"
df.to_csv(OUT_PATH, index=False)
print("Wrote:", OUT_PATH)

Wrote: ../data/clean_encounters.csv
